In [39]:
# Initial imports.
import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np
from sklearn import metrics

In [28]:
# Loading data
test_df = pd.read_csv("data/test_timeseries.csv")
test_df.head()

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,2019-01-01,2.25,100.51,9.69,14.71,13.55,13.52,17.38,10.92,...,14.63,1.20,1.50,0.79,0.71,2.74,4.01,1.23,2.78,0.0
1,1001,2019-01-02,4.94,100.48,8.65,13.05,11.83,11.74,17.76,9.54,...,13.06,1.02,1.35,0.32,1.03,2.13,3.37,0.56,2.81,NaN
2,1001,2019-01-03,20.74,100.03,8.59,12.12,11.67,11.67,13.74,10.44,...,12.12,1.83,4.23,0.34,3.88,3.41,7.49,0.72,6.77,NaN
3,1001,2019-01-04,16.17,99.47,7.93,11.80,10.38,9.59,18.19,5.44,...,11.72,3.75,5.25,2.34,2.91,6.78,9.61,4.90,4.71,NaN
4,1001,2019-01-05,0.01,100.39,5.32,7.91,4.73,4.71,14.37,3.68,...,7.75,2.32,2.90,1.69,1.21,4.42,5.87,3.15,2.72,NaN


In [29]:
# Drop NaN drought scores
test_df.dropna(inplace=True)
# Set NaN drought scores to the most recent
# test_df.fillna(method='ffill', inplace=True)
test_df.head()

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,2019-01-01,2.25,100.51,9.69,14.71,13.55,13.52,17.38,10.92,...,14.63,1.20,1.50,0.79,0.71,2.74,4.01,1.23,2.78,0.0
7,1001,2019-01-08,0.05,100.57,8.75,13.15,12.01,11.98,20.04,9.36,...,13.05,2.33,3.40,1.68,1.72,5.13,6.51,3.67,2.85,0.0
14,1001,2019-01-15,0.00,101.07,4.04,3.91,0.94,0.96,9.40,1.02,...,3.91,1.73,2.40,1.36,1.04,3.28,4.13,2.39,1.75,0.0
21,1001,2019-01-22,0.03,101.14,4.96,6.03,3.81,3.08,12.43,-1.63,...,6.08,3.68,4.57,2.49,2.07,6.65,8.56,5.01,3.55,0.0
28,1001,2019-01-29,5.79,100.36,3.69,2.23,-0.41,-1.03,7.33,-5.19,...,2.18,3.10,4.52,1.71,2.81,5.62,6.96,3.60,3.35,0.0


In [30]:
# Define the features set.
X = test_df.copy()
X = X.drop("score", axis=1)
X = X.drop("date", axis=1)
X.head()

,fips,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE
0,1001,2.25,100.51,9.69,14.71,13.55,13.52,17.38,10.92,6.46,14.63,1.20,1.50,0.79,0.71,2.74,4.01,1.23,2.78
7,1001,0.05,100.57,8.75,13.15,12.01,11.98,20.04,9.36,10.68,13.05,2.33,3.40,1.68,1.72,5.13,6.51,3.67,2.85
14,1001,0.00,101.07,4.04,3.91,0.94,0.96,9.40,1.02,8.38,3.91,1.73,2.40,1.36,1.04,3.28,4.13,2.39,1.75
21,1001,0.03,101.14,4.96,6.03,3.81,3.08,12.43,-1.63,14.06,6.08,3.68,4.57,2.49,2.07,6.65,8.56,5.01,3.55
28,1001,5.79,100.36,3.69,2.23,-0.41,-1.03,7.33,-5.19,12.52,2.18,3.10,4.52,1.71,2.81,5.62,6.96,3.60,3.35


In [31]:
# Define the target set.
y = test_df["score"].ravel()
y[:5]

array([0., 0., 0., 0., 0.])

In [32]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [33]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [34]:
# Create a random forest classifier.
rf_model = RandomForestRegressor(n_estimators=128, random_state=1) 

In [35]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [36]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [41]:
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, predictions))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
mape = np.mean(np.abs((y_test - predictions) / np.abs(y_test)))
print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2))
print('Accuracy:', round(100*(1 - mape), 2))

Mean Absolute Error (MAE): 0.26875707529915116
Mean Squared Error (MSE): 0.227910793374645
Root Mean Squared Error (RMSE): 0.4774000349545913
Mean Absolute Percentage Error (MAPE): nan
Accuracy: nan


/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [42]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.13035395, 0.04748366, 0.16337887, 0.04441748, 0.03028304,
       0.05333723, 0.05933023, 0.05599217, 0.05070245, 0.06168139,
       0.04967446, 0.02817336, 0.03157874, 0.02868458, 0.03622582,
       0.03070193, 0.03344309, 0.02838346, 0.03617407])